In [1]:
import pandas as pd

# Read the CSV file into a DataFrame
data = pd.read_csv("M:\BIO\80_DATASCIENCE\DATA\IEM\IEM_Molis_2025_01_23_imputed_by_mean.csv")

# Display the first few rows of the DataFrame
print(data.head())


<>:4: SyntaxWarning: invalid escape sequence '\B'
<>:4: SyntaxWarning: invalid escape sequence '\B'
C:\Users\ma2591\AppData\Local\Temp\ipykernel_12660\1341828160.py:4: SyntaxWarning: invalid escape sequence '\B'
  data = pd.read_csv("M:\BIO\80_DATASCIENCE\DATA\IEM\IEM_Molis_2025_01_23_imputed_by_mean.csv")


   ROW_SUMMARY  NUMERO_BON      IPP  NUMERO_SEJOUR     DATE_PRELEVEMENT  \
0    79.220779  8104197473   3979.0    324129749.0  2024-11-05 00:00:00   
1    78.896104  8103183349   3979.0    721194796.0  2023-05-12 11:59:00   
2    78.896104  8103505493   3979.0    721194796.0  2023-11-09 10:29:00   
3    36.038961  8113882464   3979.0    721194796.0  2024-05-15 10:12:00   
4    78.896104  2050071245  15875.0    723163963.0  2023-09-04 08:00:00   

      AA1MH     AA3MH    AA5ALA     AAAAD      AAABU  ...  UOURI  UOVALGLY  \
0  4.191186  3.271037  0.025129  2.341306  24.325375  ...    0.0       0.0   
1  4.191186  3.271037  0.025129  2.341306  24.325375  ...    0.0       0.0   
2  4.191186  3.271037  0.025129  2.341306  24.325375  ...    0.0       0.0   
3  4.191186  3.271037  0.025129  2.341306  24.325375  ...    0.0       0.0   
4  4.191186  3.271037  0.025129  2.341306  24.325375  ...    0.0       0.0   

    UOVANIL  UOVANMAN  UOVANPYR  DATE_NAISSANCE  SEXE_CODE  \
0  2.654391   3.11

# PCA

Standardize data

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from datetime import datetime

df=data.copy()


# Drop the 'DATE_PRELEVEMENT' column
df = df.drop(columns=['DATE_PRELEVEMENT'])

# Convert the 'DATE_NAISSANCE' column to datetime (if it's not already)
df['DATE_NAISSANCE'] = pd.to_datetime(df['DATE_NAISSANCE'], format='%d.%m.%Y')

# Create a new 'AGE' column based on the date of birth (DATE_NAISSANCE) and today's date
today = datetime.today()
df['AGE'] = df['DATE_NAISSANCE'].apply(lambda x: today.year - x.year - ((today.month, today.day) < (x.month, x.day)))

# Drop the 'DATE_NAISSANCE' column now that we have the age
df = df.drop(columns=['DATE_NAISSANCE'])

if 'GENDER' in df.columns:
    df['GENDER'] = df['GENDER'].map({'M': 1, 'F': 0})  # Encoding 'M' as 1 and 'F' as 0

# Drop any other non-numeric columns if you don't want to encode them
df = df.select_dtypes(include=['float64', 'int64'])  # Keep only numeric columns

# Standardize the data
scaler = StandardScaler()
standardized_data = scaler.fit_transform(df)

c:\ProgramData\anaconda3\envs\dataprocessing\Lib\site-packages\sklearn\utils\extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\ProgramData\anaconda3\envs\dataprocessing\Lib\site-packages\sklearn\utils\extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
c:\ProgramData\anaconda3\envs\dataprocessing\Lib\site-packages\sklearn\utils\extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


Reduce to 10 dimensions

In [4]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


# Convert the NumPy array back to a DataFrame
df_standardized = pd.DataFrame(standardized_data)

# Drop columns with missing values
df_standardized_clean = df_standardized.dropna(axis=1)

pca = PCA(n_components=10)  # Reduce to 10 dimensions
data_pca = pca.fit_transform(df_standardized_clean)

# Convert the result to a DataFrame
pca_df = pd.DataFrame(data_pca, columns=[f'PC{i+1}' for i in range(data_pca.shape[1])])

# Show the PCA result
print(pca_df)


           PC1        PC2       PC3       PC4        PC5       PC6       PC7  \
0    -0.036206   1.511944 -0.405377 -0.124901   3.835172  0.300848  1.790405   
1    -0.049520   7.852440  1.739534  3.121112  10.402487  3.057876  3.182912   
2     0.099053  11.751375  3.480701  5.712391  11.600669  3.705491  3.808657   
3    -1.545828   1.920011 -2.316577  1.759382   5.807283 -0.628739  2.925766   
4    -0.213894  -0.648364 -0.532461 -0.534373   0.162583 -0.167065 -0.209486   
...        ...        ...       ...       ...        ...       ...       ...   
3636 -0.693451  -1.388843  0.525217  0.791163   0.060613 -0.131344  0.598510   
3637 -0.574440  -1.451705  0.825782  0.733929   0.092832  0.053575  0.449769   
3638  0.144349   1.533345 -0.391248  1.471102  -1.051192  0.609056 -0.357949   
3639 -0.487039  -1.311337  1.460279  1.686430  -1.433523  0.838259 -0.310706   
3640  0.550182  -0.115719  0.108065  0.313714   0.251676 -0.102620 -0.197576   

           PC8       PC9      PC10  
0 

Add back the date of birth, sex, numéro de bon, IPP, date prélèvement

In [5]:
# Ensure the columns exist in df_imputed_mean
columns_to_add = ['IPP', 'NUMERO_BON', 'DATE_NAISSANCE', 'SEXE_CODE', 'DATE_PRELEVEMENT', 'NUMERO_SEJOUR']

# Align the indices by using df.index, ensuring no mismatch
additional_columns = data.loc[df.index, columns_to_add]

# Concatenate the additional columns to the PCA result DataFrame
pca_df_with_additional = pd.concat([pca_df, additional_columns], axis=1)

# Show the resulting DataFrame
print(pca_df_with_additional)


           PC1        PC2       PC3       PC4        PC5       PC6       PC7  \
0    -0.036206   1.511944 -0.405377 -0.124901   3.835172  0.300848  1.790405   
1    -0.049520   7.852440  1.739534  3.121112  10.402487  3.057876  3.182912   
2     0.099053  11.751375  3.480701  5.712391  11.600669  3.705491  3.808657   
3    -1.545828   1.920011 -2.316577  1.759382   5.807283 -0.628739  2.925766   
4    -0.213894  -0.648364 -0.532461 -0.534373   0.162583 -0.167065 -0.209486   
...        ...        ...       ...       ...        ...       ...       ...   
3636 -0.693451  -1.388843  0.525217  0.791163   0.060613 -0.131344  0.598510   
3637 -0.574440  -1.451705  0.825782  0.733929   0.092832  0.053575  0.449769   
3638  0.144349   1.533345 -0.391248  1.471102  -1.051192  0.609056 -0.357949   
3639 -0.487039  -1.311337  1.460279  1.686430  -1.433523  0.838259 -0.310706   
3640  0.550182  -0.115719  0.108065  0.313714   0.251676 -0.102620 -0.197576   

           PC8       PC9      PC10     